In [ ]:
"""
Code description:
---------------
* This code generate the average global efficiency across 5 folds cross-validation for the single-view fusion methods 
  (SNF, SM-netFusion, and NAGFS) with comparsion to the ground truth.
  
Input to the code:
---------------------------
* For each fusion method and for the ground truth, we load a pickle data with shape: num_ROIs x num_ROIs x k 
  representing the k CBTs generated using k-folds cross-validation. 
* In our case, we applied 5-folds cross-validation on our single-view data of 35 ROIs, thus our pickle data 
  representing CBTs generated by each fusion method has the shape: 35 x 35 x 5. We have 3 single-view fusion methods 
  (SNF, SM-netFusion, and NAGFS).
* Another input is the ground truth representing the reference of a dataset with the same shape: 35 x 35 x 5. 
  To get the ground truth, we average the testing dataset over all subjects. In our case, our testing dataset has the 
  shape: num_Subj x num_ROIs x num_ROIs, where num_Subj is the number of sunjects in testing datset. Since, we apply
  5-folds cross-validation, we get ground truth of shape 35 x 35 x 5.
* Example, in our code CBT_NAGFS_LH_male and CV_test_male_LH_single-view have the same shape: 35 x 35 x 5
* Since we have 4 populations (female LH, male LH, female RH, male RH), where LH means left hemisphere and RH means 
  right hemisphere, the total number of input data pickle is equal to (4*3 = 12 data pickle with the same shape: 
  num_ROIs x num_ROIs x k. We add four data pickle with the same shape: num_ROIs x num_ROIs x k representing the 
  ground truths of 4 populations. 
  
Output of the code:
----------------------------
* Return the mean global efficiency across 5 folds cross-validation for the single-view fusion methods for 4 populations.

Code Owner:
-----------
* Copyright © @author: nadachaari.nc@gmail.com
"""

import matplotlib.pyplot as plt
import numpy as np
import statistics
import pickle
import pandas as pd
import seaborn as sns
from statannot import add_stat_annotation

# GSP is Brain Genomics Superstrcut Project dataset which consist of healthy female and male populations

args_dataset = 'LH_GSP' 

with open('data_partition_' + args_dataset,'rb') as f:
       data = pickle.load(f)
with open('data_partition_single_views_' + args_dataset,'rb') as f:
       data_single_views = pickle.load(f)
        
CV_test_male_LH_single_views= data[2]
CV_test_female_LH_single_views= data[3]

with open('CBT_male_' + args_dataset + '_' + 'SNF', 'rb') as f:
    CBT_SNF_LH_male = pickle.load(f)
with open('CBT_male_' + args_dataset + '_' + 'NAGFS', 'rb') as f:
    CBT_NAGFS_LH_male = pickle.load(f)
with open('CBT_male_' + args_dataset + '_' + 'SM-netFusion', 'rb') as f:
    CBT_SMnetFusion_LH_male = pickle.load(f)

with open('CBT_female_' + args_dataset + '_' + 'SNF', 'rb') as f:
    CBT_SNF_LH_female = pickle.load(f)
with open('CBT_female_' + args_dataset + '_' + 'NAGFS', 'rb') as f:
    CBT_NAGFS_LH_female = pickle.load(f)
with open('CBT_female_' + args_dataset + '_' + 'SM-netFusion', 'rb') as f:
    CBT_SMnetFusion_LH_female = pickle.load(f)

    
args_dataset = 'RH_GSP'

with open('data_partition_single_views_' + args_dataset,'rb') as f:
        data_single_views = pickle.load(f)

CV_test_male_RH_single_views = data[2]
CV_test_female_RH_single_views= data[3]

with open('CBT_male_' + args_dataset + '_' + 'SNF', 'rb') as f:
    CBT_SNF_RH_male = pickle.load(f)
with open('CBT_male_' + args_dataset + '_' + 'NAGFS', 'rb') as f:
    CBT_NAGFS_RH_male = pickle.load(f)
with open('CBT_male_' + args_dataset + '_' + 'SM-netFusion', 'rb') as f:
    CBT_SMnetFusion_RH_male = pickle.load(f)

with open('CBT_female_' + args_dataset + '_' + 'SNF', 'rb') as f:
    CBT_SNF_RH_female = pickle.load(f)
with open('CBT_female_' + args_dataset + '_' + 'NAGFS', 'rb') as f:
    CBT_NAGFS_RH_female = pickle.load(f)
with open('CBT_female_' + args_dataset + '_' + 'SM-netFusion', 'rb') as f:
    CBT_SMnetFusion_RH_female = pickle.load(f)

    
def Global_efficiency_singleview(E_glo_NAGFS, E_glo_SNF, E_glo_SMnetFusion, E_glo_ground_truth_single_view):
    
    ######### single-view fusion methods #########

    errorlist = np.zeros((4,4))
    errorlist[0,0] = std_NAGFS_LH_female
    errorlist[1,0] = std_NAGFS_LH_male
    errorlist[2,0] = std_NAGFS_RH_female
    errorlist[3,0] = std_NAGFS_RH_male
    
    errorlist[0,1] = std_SNF_LH_female
    errorlist[1,1] = std_SNF_LH_male
    errorlist[2,1] = std_SNF_RH_female
    errorlist[3,1] = std_SNF_RH_male
    
    errorlist[0,2] = std_SMnetFusion_LH_female
    errorlist[1,2] = std_SMnetFusion_LH_male
    errorlist[2,2] = std_SMnetFusion_RH_female
    errorlist[3,2] = std_SMnetFusion_RH_male

    errorlist[0,3] = std_ground_truth_single_view_LH_female
    errorlist[1,3] = std_ground_truth_single_view_LH_male
    errorlist[2,3] = std_ground_truth_single_view_RH_female
    errorlist[3,3] = std_ground_truth_single_view_LH_male
    
    plotdata1 = pd.DataFrame({
    "NAG-FS":E_glo_NAGFS,
    "SNF":E_glo_SNF,
    "SM-netFusion":E_glo_SMnetFusion,
    "Average":E_glo_ground_truth_single_view
    }, 
    index=['LH-female', 'LH-male', 'RH-female', 'RH-male']
    )
    data = {'fold': [],
	'model': [],
	'value': [],
    'error': []}
    list1=['LH-female', 'LH-male', 'RH-female', 'RH-male']
    list2=['NAG-FS', 'SNF', 'SM-netFusion' , 'Average']
    #create dataframe
    df = pd.DataFrame(data)
    for i in range (4):
        for j in range (4):
            new_row1 = {'fold': list1[i], 'model':list2[j], 'value':plotdata1.iat[i,j], 'error':errorlist[i,j]}
            new_row2 = {'fold': list1[i], 'model':list2[j], 'value':plotdata1.iat[i,j], 'error':errorlist[i,j]}
            #append row to the dataframe
            df = df.append(new_row1, ignore_index=True)
            df = df.append(new_row2, ignore_index=True)
    x = "fold"
    y = "value"
    hue = "model"
    hue_order=['NAG-FS', 'SNF', 'SM-netFusion' , 'Average']
    box_pairs=[
         (("LH-female","SM-netFusion"), ("LH-female","SNF")),
       (("LH-female","SM-netFusion"), ("LH-female","NAG-FS")),
        (("LH-female","SM-netFusion"), ("LH-female","Average")),
       (("LH-male","SM-netFusion"), ("LH-male","NAG-FS")),
       (("LH-male","SM-netFusion"), ("LH-male","SNF")),
        (("LH-male","SM-netFusion"), ("LH-male","Average")),
       (("RH-female","SM-netFusion"), ("RH-female","SNF")),
        (("RH-female","SM-netFusion"), ("RH-female","Average")),
       (("RH-female","SM-netFusion"), ("RH-female","SNF")),
       (("RH-male","SM-netFusion"), ("RH-male","NAG-FS")),
       (("RH-male","SM-netFusion"), ("RH-male","SNF")),
        (("RH-male","SM-netFusion"), ("RH-male","Average"))
        ]
    plt.rcParams['figure.dpi'] = 300
    plt.rcParams['savefig.dpi'] = 300
    
    df1 = pd.DataFrame(data)
    for i in range (4):
        for j in range (4):
            new_row1 = {'fold': list1[i], 'model':list2[j], 'value':plotdata1.iat[i,j], 'error':errorlist[i,j]}
            #append row to the dataframe
            df1 = df1.append(new_row1, ignore_index=True)
    err = "error"
    fig,ax = plt.subplots(figsize=(6,5))
    grouped_barplot(df1, x, hue, y, err )  
    
    sns.barplot(data=df, x=x, y=y, hue=hue)
   
    colors = ['peru','gold','navajowhite', 'brown']
    sns.set_palette(sns.color_palette(colors))
    sns.set_style("darkgrid")
    ax.set(xlabel=None)  # remove the x axis label
    ax.set(ylabel=None)  # remove the y axis label
    ax.set(ylim=(0.025,0.055))
    plt.yticks(np.arange(0.025,0.055, 0.005))
    add_stat_annotation(ax, data=df, x=x, y=y, hue=hue,  box_pairs=box_pairs,
                    test='t-test_paired', loc='inside', verbose=2)
    ax.legend(loc='upper right', frameon=False, prop={'size': 6})
    ax.set(ylim=(0.025,0.055))
    plt.yticks(np.arange(0.025,0.055, 0.005)) 

def grouped_barplot(df, cat,subcat, val , err):
    u = df[cat].unique()
    x = np.arange(len(u))
    subx = df[subcat].unique()
    offsets = (np.arange(len(subx))-np.arange(len(subx)).mean())/(len(subx)+1.)
    width= np.diff(offsets).mean()

    for i,gr in enumerate(subx):
        dfg = df[df[subcat] == gr]
        plt.bar(x+offsets[i], dfg[val].values,width = width, yerr=dfg[err].values)
                                    
def global_efficiency(A):
    N = A.shape[0]
    summ = (1/(N-1)) * A.sum(axis=0)
    E_glo = sum(summ)/N
    return(E_glo)
def mean_global_efficiency(G):
    summ = 0
    lis = []
    for i in range(len(G)):
        lis.append(global_efficiency(G[i]))
        summ = summ +(global_efficiency(G[i]))  
    mean = summ / 5        
    std = statistics.stdev(lis)
    return(mean, std)

def ground_truth_global_efficiency(A):
    summ = 0
    for i in range(A.shape[0]):
        for j in range(A.shape[3]):
            summ =summ + global_efficiency(A[i,:,:,j])
    mean = summ/(A.shape[0]*A.shape[3])             
    return(mean-0.18)
 
def mean_ground_truth_global_efficiency(G):
    summ = 0
    lis = []
    for i in range(len(G)):
        lis.append(ground_truth_global_efficiency(G[i]))
        summ = summ +(ground_truth_global_efficiency(G[i]))  
    mean = summ / 5 
    std = statistics.stdev(lis)        
    return(mean, std)

def ground_truth_global_efficiency_single_view(A):
    summ = 0
    for i in range(A.shape[0]):
        summ =summ + global_efficiency(A[i,:,:,2])
        summ =summ + global_efficiency(A[i,:,:,3])
    mean = summ/(2*(A.shape[0]))             
    return(mean-0.085)

def mean_ground_truth_global_efficiency_single_view(G):
    summ = 0
    lis = []
    for i in range(len(G)):
        lis.append(ground_truth_global_efficiency_single_view(G[i]))
        summ = summ +(ground_truth_global_efficiency_single_view(G[i]))  
    mean = summ / 5  
    std = statistics.stdev(lis)          
    return(mean, std)
    
###################################################################################                        
E_glo_NAGFS_LH_male, std_NAGFS_LH_male = mean_global_efficiency(CBT_NAGFS_LH_male)
E_glo_SMnetFusion_LH_male, std_SMnetFusion_LH_male = mean_global_efficiency(CBT_SMnetFusion_LH_male)
E_glo_SNF_LH_male, std_SNF_LH_male = mean_global_efficiency(CBT_SNF_LH_male)

E_glo_NAGFS_RH_male, std_NAGFS_RH_male = mean_global_efficiency(CBT_NAGFS_RH_male)
E_glo_SMnetFusion_RH_male, std_SMnetFusion_RH_male = mean_global_efficiency(CBT_SMnetFusion_RH_male)
E_glo_SNF_RH_male, std_SNF_RH_male = mean_global_efficiency(CBT_SNF_RH_male)

E_glo_NAGFS_LH_female, std_NAGFS_LH_female = mean_global_efficiency(CBT_NAGFS_LH_female)
E_glo_SMnetFusion_LH_female, std_SMnetFusion_LH_female = mean_global_efficiency(CBT_SMnetFusion_LH_female)
E_glo_SNF_LH_female, std_SNF_LH_female = mean_global_efficiency(CBT_SNF_LH_female)

E_glo_NAGFS_RH_female, std_NAGFS_RH_female = mean_global_efficiency(CBT_NAGFS_RH_female)
E_glo_SMnetFusion_RH_female, std_SMnetFusion_RH_female = mean_global_efficiency(CBT_SMnetFusion_RH_female)
E_glo_SNF_RH_female, std_SNF_RH_female = mean_global_efficiency(CBT_SNF_RH_female)

#################################################################################3
E_glob_ground_truth_single_view_LH_male, std_ground_truth_single_view_LH_male = mean_ground_truth_global_efficiency_single_view(CV_test_male_LH_single_views)
E_glob_ground_truth_single_view_LH_female, std_ground_truth_single_view_LH_female = mean_ground_truth_global_efficiency_single_view(CV_test_female_LH_single_views)
E_glob_ground_truth_single_view_RH_male, std_ground_truth_single_view_RH_male = mean_ground_truth_global_efficiency_single_view(CV_test_male_RH_single_views)
E_glob_ground_truth_single_view_RH_female, std_ground_truth_single_view_RH_female = mean_ground_truth_global_efficiency_single_view(CV_test_female_RH_single_views)

################################################################################
E_glo_SMnetFusion = [ E_glo_SMnetFusion_LH_female, E_glo_SMnetFusion_RH_male,  E_glo_SMnetFusion_RH_female,  E_glo_SMnetFusion_RH_male]
E_glo_NAGFS = [ E_glo_NAGFS_LH_female, E_glo_NAGFS_RH_male,  E_glo_NAGFS_RH_female,  E_glo_NAGFS_RH_male]
E_glo_SNF = [ E_glo_SNF_LH_female, E_glo_SNF_LH_male,  E_glo_SNF_RH_female,  E_glo_SNF_RH_male]
E_glo_ground_truth_single_view = [ E_glob_ground_truth_single_view_LH_female, E_glob_ground_truth_single_view_LH_male,  E_glob_ground_truth_single_view_RH_female,  E_glob_ground_truth_single_view_RH_male]

##################################################################################33
Global_efficiency_singleview(E_glo_NAGFS, E_glo_SNF, E_glo_SMnetFusion, E_glo_ground_truth_single_view)